In [1]:
import preprocessing as preproc
import helpers
from tqdm import tqdm
import time
import text_representation as text_repr
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Embedding, Activation, Flatten, GlobalMaxPooling1D
from keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
start_time = time.time()

full_dataset = False
glove_pretrained = False

train_data, y, test_data = helpers.get_processed_data(full_dataset=full_dataset)
num_words = text_repr.define_num_words(train_data, full_dataset)
tweet_max_length = text_repr.define_tweet_max_len(train_data, full_dataset)

38648: num of words , 30000: will be used
74: max sentence length found, 80: will be used


In [3]:
tokenizer = text_repr.fit_tokenizer(train_data, num_words)

train_data_seq = tokenizer.texts_to_sequences(train_data)
test_data_seq = tokenizer.texts_to_sequences(test_data)


# train_data_seq = text_repr.create_sequences(train_data, num_words)
# test_data_seq = text_repr.create_sequences(test_data, num_words)

train_data_seq = pad_sequences(train_data_seq, maxlen=tweet_max_length)
test_data_seq = pad_sequences(test_data_seq, maxlen=tweet_max_length)

In [4]:
train_data_seq.shape

(200000, 80)

In [5]:
test_data_seq.shape

(10000, 80)

In [6]:
filepath_acc = "../data/intermediate/drop2.weights.best.hdf5"
checkpoint_acc = ModelCheckpoint(filepath_acc, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [13]:
model_cnn_02 = Sequential()
# e = Embedding(num_words, 200, weights=[embedding_matrix], input_length=tweet_max_length, trainable=True)
e = Embedding(num_words, 200, input_length=tweet_max_length)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dropout(0.2))
model_cnn_02.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=5e-4, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)

model_cnn_02.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(train_data_seq, y, test_size=0.20, random_state=42)

model_cnn_02.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32, verbose=1, callbacks=[checkpoint_acc])

Train on 160000 samples, validate on 40000 samples
Epoch 1/5
160000/160000 [==============================] - 375s 2ms/step - loss: 0.3793 - acc: 0.8218 - val_loss: 0.3589 - val_acc: 0.8364
Epoch 2/5
160000/160000 [==============================] - 373s 2ms/step - loss: 0.3164 - acc: 0.8605 - val_loss: 0.3503 - val_acc: 0.8411
Epoch 3/5
160000/160000 [==============================] - 369s 2ms/step - loss: 0.2934 - acc: 0.8731 - val_loss: 0.3488 - val_acc: 0.8433
Epoch 4/5
160000/160000 [==============================] - 386s 2ms/step - loss: 0.2781 - acc: 0.8810 - val_loss: 0.3487 - val_acc: 0.8434
Epoch 5/5
160000/160000 [==============================] - 380s 2ms/step - loss: 0.2662 - acc: 0.8873 - val_loss: 0.3480 - val_acc: 0.8447


In [8]:
# del model_cnn_02 0.84

In [9]:
# del model

In [10]:
model = load_model("../data/intermediate/drop2.weights.best.hdf5")

In [11]:
scores = []
for i in range(1,4):
    X_train, X_test, y_train, y_test = train_test_split(train_data_seq, y, test_size=10000, random_state=i)
    score = model.evaluate(X_test,y_test,verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    scores.append(score[1])
print('Average score:', sum(scores)/len(scores))

10000/10000 [==============================] - 1s 124us/step
acc: 87.00%
10000/10000 [==============================] - 1s 114us/step
acc: 87.46%
10000/10000 [==============================] - 1s 114us/step
acc: 87.79%
Average score: 0.8741666666666666


In [12]:
#predict to submit
helpers.predict_and_save(model, test_data_seq, 'CNN')